# Learning Langchain (The book)
___

The main task of the software engineer working with LLMs is not to train an LLM, or even to fine-tune one (usually), but rather to take an existing LLM and work out how to get it to accomplish the task you need for your application. Adapting an existing LLM for your task is called `prompt engineering`

`prompt engineering with LangChain` - how to use LangChain to get LLMs to do what do what you have in mind.

## Prompt technique
___

* Zero-Shot Prompting
* Chain-of-Thought(CoT)
* Retrieval-Augmented Generation - in real applications should be combined with CoT
* Tool Calling - consist of prepending the prompt with a list of external functions the LLM can make use of, along with descriptions of what is good for nd instructions on how to use one (or more) of these functions.The developer of the application - should parse the output and call the appropriate functions.
* Few-Shot Prompting

Important things to keep in mind when prompting LLMs: each prompting technique is most useful when used in combination with (some of) the others.

## Retrieval-Augmented Generation (RAG)

**RAG** é uma técnica que combina **recuperação de informações** com **geração de texto**. Em vez de depender apenas do conhecimento pré-treinado do LLM, o RAG busca informações relevantes em uma base de dados externa e usa essas informações como contexto para gerar respostas mais precisas e atualizadas.

### Funcionamento do RAG:
1. **Indexação**: Documentos são vetorizados e armazenados
2. **Recuperação**: Query do usuário busca documentos similares
3. **Geração**: LLM usa documentos recuperados como contexto

`LLM interface simply takes a string prompt as input, sends the input to the model provider, and then returns the model prediction as output.`

## Capítulo 1
___

a - Chamada a um llm

In [63]:
import os

from dotenv import find_dotenv, load_dotenv
from langchain_core.prompts import ChatPromptTemplate  # type: ignore
from langchain_openai import ChatOpenAI  # type: ignore

# * Carrega as variáveis de ambiente
_ = load_dotenv(find_dotenv())

# * Verifica se a API key está configurada
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

# * Configura o modelo com parâmetros específicos
model: ChatOpenAI = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0,  # Controla a criatividade das respostas
)  # type: ignore

response = model.invoke("The Sky is ?")
print(response.content)

blue


b - Chatmodel

`HumanMessage`

In [64]:
from langchain_core.messages import HumanMessage
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI()
prompt = [HumanMessage("What is the capital of Brasil?")]
response = model.invoke(prompt)
print(response.content)

The capital of Brasil is Brasília.


c - System

`SystemMessage`

In [65]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI()
system_msg = """You are a helpful assistant that responds to questions with three exclamations marks.
"""
human_msg = HumanMessage("What is the capital of France?")
response = model.invoke([system_msg, human_msg])
print(response.content)

Paris!!!


Prompt instructions significantly influences the model's output. Prompts help the model understand context and generate relevant answers to queries.

LangChain provides prompt template interfaces that make it easy to construct prompts with dynamic inputs

In [66]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Answer the questions based on the context below.
If the question cannot be answered using the information provided, answer with "I don't know".

Context: {context}

Question: {question}""")

response = template.invoke(
    {
        "context": "The most recent advancements in NLP are being driven by Large Language Model (LLM). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's 'transformers' library, or by utilizing OpenAI and cohere's offerings through the `openai` and `cohere` libraries, respectively.",
        "question": "Which model providers offers LLMs?",
    }
)
print(response)

text='Answer the questions based on the context below.\nIf the question cannot be answered using the information provided, answer with "I don\'t know".\n\nContext: The most recent advancements in NLP are being driven by Large Language Model (LLM). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face\'s \'transformers\' library, or by utilizing OpenAI and cohere\'s offerings through the `openai` and `cohere` libraries, respectively.\n\nQuestion: Which model providers offers LLMs?'


In [67]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template(
    """Responda a questão baseada no contexto a seguir. Se a questão não puder ser respondida usando a informação, responda usando 'Não sei a resposta para essa pergunta!'

Contexto: {contexto}

Pergunta: {pergunta}"""
)

# 2. Invoke corrigido - fora do template e com aspas fechadas
response = template.invoke(
    {
        "contexto": "Quais os frameworks para a criação de IAs, chatbots, mais usados para construção de agentes de IA.",
        "pergunta": "Qual é o framework mais performático e mais simples de aprender?",
    }
)

# 3. PromptValue não tem .content, usar .text ou apenas print(response)
print(response.text)

Responda a questão baseada no contexto a seguir. Se a questão não puder ser respondida usando a informação, responda usando 'Não sei a resposta para essa pergunta!'

Contexto: Quais os frameworks para a criação de IAs, chatbots, mais usados para construção de agentes de IA.

Pergunta: Qual é o framework mais performático e mais simples de aprender?


`Both template and model can be reused many times`

In [68]:
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI

template = PromptTemplate.from_template("""Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know".

Context: {context}

Question: {question}

Answer: """)

model = ChatOpenAI(model="gpt-3.5-turbo")
# `prompt` and `completion` are the results of using template and model once

prompt = template.invoke(
    {
        "context": "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
        "question": "Which model providers offer LLMs?",
    }
)

response = model.invoke(prompt)
print(response)

content='Hugging Face, OpenAI, and Cohere offer LLMs.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 132, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C07tYZMNve7m7F1WJWItcd3KuXOdR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--0d9513c6-c043-433a-8a43-b082338c1872-0' usage_metadata={'input_tokens': 132, 'output_tokens': 15, 'total_tokens': 147, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [69]:
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI

template = PromptTemplate.from_template(
    """Responda a questão baseada no contexto a seguir. Se a questão não puder ser respondida usando a informação, responda usando 'Não sei a resposta para essa pergunta!'

Contexto: {contexto}

Pergunta: {pergunta}"""
)
model = ChatOpenAI(model="gpt-4.1")

# 2. Invoke corrigido - fora do template e com aspas fechadas
prompt = template.invoke(
    {
        "contexto": "Quais os frameworks para a criação de IAs, chatbots, mais usados para construção de agentes de IA.",
        "pergunta": "Qual é o framework mais performático e mais simples de aprender?",
    }
)

# 3. PromptValue não tem .content, usar .text ou apenas print(response)
response = model.invoke(prompt)
print(response)

content='Não sei a resposta para essa pergunta!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 85, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-C07tZAYbUcPzg4OQ1xQuBpro9lEdr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--f24761cf-de75-4448-a62a-e641b2954c72-0' usage_metadata={'input_tokens': 85, 'output_tokens': 8, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [70]:
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI

template = PromptTemplate.from_template(
"""Responda a pergunta baseada no contexto a seguir. Se a pergunta não puder ser respondida usando a informação,
previamente fornecida, responda com um 'Não sei a resposta para essa pergunta!'

Contexto: {contexto}
Pergunta: {pergunta}

""")
model = ChatOpenAI(model="gpt-4.1")
prompt = template.invoke(
    {
        "contexto": "Considerando a programação orientada a objetos no Python, o que é um objeto",
        "pergunta": "Como criar uma classe em Python usando o conceito de encapsulamento?",
    }
)

response = model.invoke(prompt)
print(response.content)

Não sei a resposta para essa pergunta!


Building an AI chat application, the `ChatPromptTemplate` can be used instead to provide dynamic inputs based on the role of the chat message:

In [71]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            'Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don\'t know".',
        ),
        ("human", "Context: {context}"),
        ("human", "Question: {question}"),
    ]
)

response = template.invoke(
    {
        "context": "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
        "question": "Which model providers offer LLMs?",
    }
)

print(response)

messages=[SystemMessage(content='Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don\'t know".', additional_kwargs={}, response_metadata={}), HumanMessage(content="Context: The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Question: Which model providers offer LLMs?', additional_kwargs={}, response_metadata={})]


In [72]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI

# both `template` and `model` can be reused many times

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            'Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don\'t know".',
        ),
        ("human", "Context: {context}"),
        ("human", "Question: {question}"),
    ]
)

model = ChatOpenAI()

# `prompt` and `completion` are the results of using template and model once

prompt = template.invoke(
    {
        "context": "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
        "question": "Which model providers offer LLMs?",
    }
)

print(model.invoke(prompt))

content='OpenAI and Cohere.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 137, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C07taxiItparR9kKemjMdGcIiheiS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--bf198af0-547e-4ff9-9bb8-cb979a680a90-0' usage_metadata={'input_tokens': 137, 'output_tokens': 6, 'total_tokens': 143, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


1. Padrão de Prompts Dinâmicos - Sim, é um padrão estabelecido!
O código que você está vendo segue um padrão muito comum e recomendado no LangChain:
Por que este padrão é importante?
```python
template = ChatPromptTemplate.from_messages([
    ("system", "Instrução do sistema..."),
    ("human", "Entrada do usuário: {variavel}"),
])
```
Reutilização: O template pode ser usado múltiplas vezes com diferentes dados
* Separação de responsabilidades: O prompt (template) é separado da execução (model)
* Manutenibilidade: Fácil de modificar e testar prompts
* Escalabilidade: Pode ser usado em pipelines complexos

Quando usar:
* Definir o papel(role)/comportamento(behave) do modelo.
* Estabelecer regras e limitações
* Configurar o contexto geral da conversa
* Definir o tom e estilo de resposta.


```python
("system", "Você é um tutor de programação Python. Sempre explique conceitos de forma simples e forneça exemplos práticos.")
```

`HumanMessage` (Mensagem Humana)

```python
("human", "Pergunta: {question}")
```

Quando Usar:

* Entradas do usuário
* Perguntas específicas
* Dados que precisam ser processados
* Contexto que o usuário fornece

```python
("human", "Explique o conceito de list comprehension em Python")
```

`AIMessage (Mensagem da IA)`
```python
("ai", "Resposta da IA...")
```

Quando Usar:

* Simular conversar anteriores
* Fornecer exemplos de respostas
* Criar contextos de few-shot learning
* Manter histórico de conversas

Exemplo prático:
```python
("ai", "List comprehension é uma forma concisa de criar listas em Python. Exemplo: [x*2 for x in range(5)]")
```

`Estrutura Recomendada para Prompts Dinâmicos`
___
```python
# 1. Definir o template com tipos apropriados
template = ChatPromptTemplate.from_messages([
    ("system", "Defina o papel/contexto do modelo"),
    ("human", "Forneça o contexto: {context}"),
    ("human", "Faça a pergunta: {question}"),
])

# 2. Criar o modelo
model = ChatOpenAI()

# 3. Invocar com dados dinâmicos
prompt = template.invoke({
    "context": "seu contexto aqui",
    "question": "sua pergunta aqui"
})

# 4. Obter a resposta
response = model.invoke(prompt)
```

`**Boas Práticas para Desenvolvedores de Agentes**`
___

1.Sempre use templates para prompts dinâmicos

    -Evite strings hardcoded
    -Facilita testes e modificações

2.Separe claramente os tipos de mensagens

    -System: configuração/contexto
    -Human: entrada do usuário
    -AI: respostas/exemplos

3.Use variáveis para dados dinâmicos

    -{context}, {question}, {user_input}
    -Facilita a reutilização

4.Mantenha prompts modulares

    -Cada template com responsabilidade específica
    -Combine templates para casos complexos

**Versão básica**
```python
template = ChatPromptTemplate.from_messages([
    ("system", "Responda baseado no contexto."),
    ("human", "Contexto: {context}"),
    ("human", "Pergunta: {question}"),
])
```
**Versão avançada(para agentes)**
```python
template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente especializado em análise de dados."),
    ("human", "Contexto: {context}"),
    ("human", "Pergunta: {question}"),
    ("ai", "Vou analisar o contexto e responder sua pergunta."),
    ("human", "Por favor, seja específico e forneça exemplos."),
])
```

In [73]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
# Adicionamos o Output Parser para extrair a resposta final como texto
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4.1")

template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente de IA que é especialista em filmes e séries."),
    ("human", "Contexto: {contexto}"),
    ("human", "Pergunta: {pergunta}"),
    ("ai", "Analise o contexto e responda a pergunta")
])
# Conectamos os componentes na ordem que devem ser executados.
chain = template | model | StrOutputParser()

# 4. Invocando a Chain (com o input correto)
# Agora, fornecemos um dicionário com as chaves que o template espera.
# Note que o contexto e a pergunta agora fazem sentido com o prompt do sistema.
contexto_exemplo = "Em 'O Senhor dos Anéis: A Sociedade do Anel', Frodo Bolseiro, um hobbit, herda um anel poderoso de seu tio Bilbo. Ele descobre que o anel é o Um Anel, uma arma do Lorde das Trevas Sauron, e deve embarcar em uma jornada para destruí-lo na Montanha da Perdição."
pergunta_exemplo = "Qual é o objetivo da jornada de Frodo?"

response = chain.invoke({
    "contexto": contexto_exemplo, 
    "pergunta": pergunta_exemplo
})

# 5. Imprimindo a resposta
print(response)

O objetivo da jornada de Frodo é destruir o Um Anel, levando-o até a Montanha da Perdição, onde ele foi forjado, para que o anel seja destruído e, assim, impedir que o Lorde das Trevas Sauron recupere seu poder e domine a Terra-média.


Para lembrar:

 - `SystemMessage` = "Quem você é"
 - `HumanMessage` = "O que o usuário quer"
 - `AIMessage` = "Como você responde"

In [74]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0)

messages = [
    SystemMessage(
        content="Você é um assistente prestativo, especialista em Matemática."
    ),
    HumanMessage(content="Explique o que é um modelo de machine learning, em até 30 palavras"),
]

response = chat.invoke(messages)
print(response.content)

Um modelo de machine learning é um algoritmo que aprende padrões a partir de dados para fazer previsões ou tomar decisões automatizadas em situações futuras.


`PromptTemplate` simples é só para texto, `ChatPromptTemplate` é para conversas com roles(especialista em Python, ou matemática, etc).

In [75]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("Você é um especialista em {tópico}"),
        HumanMessagePromptTemplate.from_template("Explique {assunto} em 35 palavras"),
    ]
)

# Template
formatted_prompt = prompt.format_messages(tópico="framework", assunto="langchain")

response = chat.invoke(formatted_prompt)
print(response.content)

Langchain é um framework de desenvolvimento de aplicativos descentralizados (dApps) que utiliza a linguagem de programação Rust. Ele facilita a criação de dApps seguras e escaláveis, integrando-se com blockchains como Ethereum.


In [76]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Template com papel específico
template = ChatPromptTemplate.from_messages([ # type: ignore
    ("system", "Você é um tutor de programação Python especializado em explicar conceitos complexos de forma simples."),
    ("human", "Conceito: {conceito}"),
    ("human", "Explique este conceito em {palavras} palavras ou menos"),
])

model = ChatOpenAI()

# Uso do template
response = model.invoke(template.invoke({# type: ignore
    "conceito": "decorators em Python",
    "palavras": "50"
}))

print(response.content)# type: ignore

Em Python, um decorator é uma função que recebe outra função como argumento e retorna uma nova função modificada. Decorators são usados para adicionar funcionalidades a funções existentes sem modificar seu código diretamente, permitindo reutilização de código e separação de preocupações.


## Getting Specific Formats out of LLMs
___

The most common format to generate with LLMs is JSON, which can be sent over the to your frontend code or be saved to a databased.

1. First task, define the output schema (from LLM). Then include that schema in the prompt , along with the text you want to use as the source.
2. In order to define a `schema`, this is easiest to do with `Pydantic`(*library* used for validating data against schemas)


In [77]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel


class AnswerWithJustification(BaseModel):
    """An answer to the user's question along with justification for the answer."""

    answer: str
    """The answer to the user's question"""
    justification: str
    """Justification for the answer"""


model = ChatOpenAI(model="gpt-4.1", temperature=0)
structured_model = model.with_structured_output(AnswerWithJustification)

response = structured_model.invoke(
    "What weighs more, a pound of bricks or a pound of feathers"
)
print(response)

answer='They weigh the same.' justification='A pound is a unit of weight, so a pound of bricks and a pound of feathers both weigh exactly one pound. The material does not affect the weight if the quantity is specified as a pound.'


#### TUTORIAL: LangChain + Pandantic - Structured Output
========================================================

Este exemplo demonstra como usar Pydantic com LangChain para:
1. Validar dados de entrada e saída
2. Garantir formato estruturado das respostas
3. Tornar o código mais robusto e type-safe


In [78]:
import json
from datetime import datetime

from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [79]:
class AnswerWithJustification(BaseModel):
    """
    Modelo que representa uma resposta estruturada com justificativa.

      Por que usar Pydantic aqui?
    - Validação automática de tipos
    - Documentação clara dos campos
    - Serialização JSON automática
    - Integração nativa com LangChain
    """

    answer: str = Field(
        description="Resposta clara e concisa para a pergunta",
        min_length=1,
        max_length=500,
    )

    justification: str = Field(
        description="Explicação detalhada do raciocínio", min_length=10, max_length=100
    )

    confidence_level: float | None = Field(
        default=None, description="Nível de confiança (0-1) na resposta", ge=0.0, le=1.0
    )

    timestamp: str | None = Field(
        default_factory=lambda: datetime.now().isoformat(),
        description="Timestamp da resposta",
    )

    def display_response(self) -> str:
        """
        Método para exibir a resposta de forma mais legível.

        🔧 BENEFÍCIO: Evita o scroll infinito no output!
        """
        output = f"""
┌─ 💬 RESPOSTA ─────────────────────────────────────────┐
│ {self.answer}
├─ 🤔 JUSTIFICATIVA ───────────────────────────────────┤
│ {self.justification}
"""

        if self.confidence_level:
            confidence_bar = "█" * int(self.confidence_level * 10)
            output += f"""├─ 📊 CONFIANÇA ───────────────────────────────────────┤
│ {confidence_bar} {self.confidence_level:.1%}
"""

        output += f"""└─ 🕒 {self.timestamp} ──────────────────────────────────┘"""
        return output

#### 🚀 CLASSE PRINCIPAL PARA DEMONSTRAÇÃO

`class LangChainPydanticDemo`

In [80]:
class LangChainPydanticDemo:
    """
    Conceitos Importantes:
    1. Structured Output: LangChain força o LLM a retornar dados no formato exato do
    modelo Pydantic.

    2.Type safety: Pydantic valida automaticamente os tipos

    3.Error Handling: Falhas na validação são capturadas
    """

    def __init__(self, model_name="gpt-4.1"):
        self.model = ChatOpenAI(
            model=model_name,
            temperature=0.1,  # * baixa criatividade para respostas consistentes
        )

        # * 🔑 PONTO CHAVE: with_structured_output() força o formato
        self.structured_model = self.model.with_structured_output(
            AnswerWithJustification
        )

    def ask_question(self, question: str) -> AnswerWithJustification:
        """
        Faz uma pergunta e retorna resposta estruturada.

        🎯 BENEFÍCIOS do Structured Output:
        - Resposta sempre no formato esperado
        - Validação automática dos dados
        - Facilita integração com APIs e bancos de dados
        """
        try:
            response = self.structured_model.invoke(question)
            return response

        except Exception as e:
            # * Em caso de erro, retorna resposta padrão
            return AnswerWithJustification(
                answer="Erro ao processar pergunta",
                justification=f"Erro técnico: {e!s}",
                confidence_level=0.0,
            )

    def demo_multiple_questions(self):
        """
        Demonstra o uso com múltiplas perguntas para mostrar consistência.
        """
        questions = [
            "O que pesa mais: 1kg de chumbo ou 1kg de algodão?",
            "Por que o céu é azul?",
            "Qual a diferença entre Python e JavaScript?",
        ]

        print("🎓 DEMONSTRAÇÃO: Múltiplas perguntas com formato consistente\n")

        for i, question in enumerate(questions, 1):
            print(f"❓ PERGUNTA {i}: {question}")
            response = self.ask_question(question)
            print(response.display_response())
            print("\n" + "=" * 60 + "\n")

#### 📖 EXEMPLOS PRÁTICOS DE USO

In [81]:
def exemplo_basico():
    """Exemplo básico de uso do structured output."""
    print("🔵 EXEMPLO 1: Uso Básico")
    print("-" * 40)

    demo = LangChainPydanticDemo()

    question = "Explique o conceito de recursão em programação"
    response = demo.ask_question(question)

    # ✨ SAÍDA LIMPA - sem scroll infinito!
    print(response.display_response())

    # 💾 BONUS: Fácil conversão para JSON
    print("\n🔧 BONUS - Dados em JSON:")
    print(json.dumps(response.model_dump(), indent=2, ensure_ascii=False))

In [82]:
exemplo_basico()

🔵 EXEMPLO 1: Uso Básico
----------------------------------------

┌─ 💬 RESPOSTA ─────────────────────────────────────────┐
│ Recursão em programação é uma técnica onde uma função chama a si mesma, direta ou indiretamente, para resolver um problema dividindo-o em subproblemas menores do mesmo tipo.
├─ 🤔 JUSTIFICATIVA ───────────────────────────────────┤
│ A recursão é usada quando um problema pode ser decomposto em versões menores de si mesmo. Por meio d
├─ 📊 CONFIANÇA ───────────────────────────────────────┤
│ █████████ 99.0%
└─ 🕒 2024-06-18T18:00:00Z ──────────────────────────────────┘

🔧 BONUS - Dados em JSON:
{
  "answer": "Recursão em programação é uma técnica onde uma função chama a si mesma, direta ou indiretamente, para resolver um problema dividindo-o em subproblemas menores do mesmo tipo.",
  "justification": "A recursão é usada quando um problema pode ser decomposto em versões menores de si mesmo. Por meio d",
  "confidence_level": 0.99,
  "timestamp": "2024-06-18T18:00:00Z"


In [83]:
def exemplo_avancado():
    """Exemplo avançado mostrando validação."""
    print("\n🟢 EXEMPLO 2: Validação com Pydantic")
    print("-" * 40)

    # Tentativa de criar resposta inválida (para mostrar validação)
    try:
        invalid_response = AnswerWithJustification(
            answer="",  # ❌ Inválido: muito curto
            justification="Curto",  # ❌ Inválido: muito curto
            confidence_level=1.5,  # ❌ Inválido: > 1.0
        )
    except Exception as e:
        print(f"❌ VALIDAÇÃO PYDANTIC FUNCIONOU: {e}")

    # ✅ Resposta válida
    valid_response = AnswerWithJustification(
        answer="Python é uma linguagem de programação",
        justification="Python é conhecida por sua sintaxe simples e legível",
        confidence_level=0.9,
    )

    print("\n✅ RESPOSTA VÁLIDA:")
    print(valid_response.display_response())

In [84]:
exemplo_avancado()


🟢 EXEMPLO 2: Validação com Pydantic
----------------------------------------
❌ VALIDAÇÃO PYDANTIC FUNCIONOU: 3 validation errors for AnswerWithJustification
answer
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short
justification
  String should have at least 10 characters [type=string_too_short, input_value='Curto', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short
confidence_level
  Input should be less than or equal to 1 [type=less_than_equal, input_value=1.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal

✅ RESPOSTA VÁLIDA:

┌─ 💬 RESPOSTA ─────────────────────────────────────────┐
│ Python é uma linguagem de programação
├─ 🤔 JUSTIFICATIVA ───────────────────────────────────┤
│ Python é conhecida por sua sintaxe simples e legível
├─ 📊 CONFIANÇA 

In [85]:
# Demonstração completa
print("\n🎓 DEMONSTRAÇÃO COMPLETA:")
print("=" * 60)
demo = LangChainPydanticDemo()
demo.demo_multiple_questions()


🎓 DEMONSTRAÇÃO COMPLETA:
🎓 DEMONSTRAÇÃO: Múltiplas perguntas com formato consistente

❓ PERGUNTA 1: O que pesa mais: 1kg de chumbo ou 1kg de algodão?

┌─ 💬 RESPOSTA ─────────────────────────────────────────┐
│ 1 kg de chumbo pesa o mesmo que 1 kg de algodão.
├─ 🤔 JUSTIFICATIVA ───────────────────────────────────┤
│ A unidade de medida 'quilograma' (kg) representa uma quantidade fixa de massa. Portanto, 1 kg de um 
├─ 📊 CONFIANÇA ───────────────────────────────────────┤
│ ██████████ 100.0%
└─ 🕒 2024-06-18T20:00:00Z ──────────────────────────────────┘


❓ PERGUNTA 2: Por que o céu é azul?

┌─ 💬 RESPOSTA ─────────────────────────────────────────┐
│ O céu é azul porque a luz do Sol, ao passar pela atmosfera da Terra, sofre espalhamento pelas moléculas de ar, e a luz azul é espalhada em todas as direções com mais intensidade do que outras cores.
├─ 🤔 JUSTIFICATIVA ───────────────────────────────────┤
│ A luz branca do Sol é composta por várias cores, cada uma com um comprimento de onda dif

🎓 LIÇÕES APRENDIDAS:

1. **Pydantic + LangChain = Dados Estruturados**
   - LLM retorna sempre no formato correto
   - Validação automática de tipos e valores

2. **Benefícios para Produção**
   - Code completion no IDE
   - Documentação automática
   - Integração fácil com APIs

3. **Melhor UX**
   - Saída formatada e legível
   - Sem scroll infinito
   - Informações organizadas

4. **Type Safety**
   - Erros capturados em desenvolvimento
   - Código mais robusto
   - Manutenção facilitada

**Por que Pydantic é Fundamental no LangChain**:

1. Structured Output: Força o LLM a retornar dados no formato exato
2. Validação Automática: Tipos e valores são verificados automaticamente
3. Integração Nativa: LangChain foi projetado para trabalhar com Pydantic
4. Produção Ready: Facilita APIs, banco de dados e frontend

#### Other Machine-Readable Formats with Output Parsers
___

We can also use an LLM or chat model to produce output in other formats, such as CSV or XML. Using output parsers  that are classes that help us structure large language model responses.

    - Providing format instructions
    - Validating and parsing output

In [86]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

response = parser.invoke("apple, banana, cherry")
print(response)

['apple', 'banana', 'cherry']


In [87]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

1. Classe Pydantic

In [88]:
class UserInfo(BaseModel):
    name: str
    age: int
    email: str

2. Parser

In [89]:
parser = PydanticOutputParser(pydantic_object=UserInfo)

3. Prompt com instrução de saída estruturada

In [90]:
prompt = PromptTemplate(
    template = """Dado o seguinte texto: '{text}', gere um JSON com as seguintes informações: {format_instructions}""",
    input_variables = ["text"],
    partial_variables = {"format_instructions": parser.get_format_instructions()}
)

4. LLM

In [91]:
llm = ChatOpenAI(model="gpt-4.1", temperature=0.4)

In [92]:
input_text = """O nome dela é Ana Paula, tem 32 anos e o email dela é ana.paula@gmail.com, e ela mora em São Paulo"""

chain = prompt | llm | parser
response = chain.invoke({"text": input_text})
print(response)

name='Ana Paula' age=32 email='ana.paula@gmail.com'


In [93]:
from langchain.prompts import PromptTemplate

csv_prompt = PromptTemplate.from_template(
    "Converta a seguinte lista de produtos em formato CSV com colunas: nome, preço, categoria.\n\n{text}"
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

input_text = """
1. Arroz, 20 reais, Alimentos
2. Sabonete, 5 reais, Higiene
3. Caderno, 15 reais, Papelaria
"""

csv_result = llm.invoke(csv_prompt.format(text=input_text))
print(csv_result.content)

nome, preço, categoria
Arroz, 20 reais, Alimentos
Sabonete, 5 reais, Higiene
Caderno, 15 reais, Papelaria


In [94]:
# Modelo simples para validar como Python dict
class BookInfo(BaseModel):
    title: str
    author: str
    year: int

book_parser = PydanticOutputParser(pydantic_object=BookInfo)

book_prompt = PromptTemplate(
    template="Extraia as informações do livro e retorne no formato:\n{format_instructions}\n\nTexto: {text}",
    input_variables=["text"],
    partial_variables={"format_instructions": book_parser.get_format_instructions()}
)

chain = book_prompt | llm | book_parser

text = "O livro '1984' foi escrito por George Orwell e publicado em 1949."
book_info = chain.invoke({"text": text})
print(book_info)
print(book_info.model_dump())  # Convertendo para dict usando model_dump()


title='1984' author='George Orwell' year=1949
{'title': '1984', 'author': 'George Orwell', 'year': 1949}


In [95]:
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    "Extraia o título, autor e ano do seguinte texto:\n\nTexto: {text}"
)

chain = prompt | llm | StrOutputParser()
text = "O livro 'Dom Casmurro' foi escrito por Machado de Assis em 1899."
response = chain.invoke({"text": text})
print(response)


Título: Dom Casmurro
Autor: Machado de Assis
Ano: 1899


In [96]:
class Book(BaseModel):
    title: str
    author: str
    year: int

parser = PydanticOutputParser(pydantic_object=Book)

prompt = PromptTemplate(
    template="Extraia os dados do livro e retorne em JSON no seguinte formato:\n{format_instructions}\n\nTexto: {text}",
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | llm | parser
book = chain.invoke({"text": text})
print(book.model_dump())  # Convertendo para dict usando model_dump()


{'title': 'Dom Casmurro', 'author': 'Machado de Assis', 'year': 1899}


| Parser                 | Quando usar                                       |
| ---------------------- | ------------------------------------------------- |
| `StrOutputParser`      | Para capturar texto puro, rascunhos, debugging    |
| `PydanticOutputParser` | Quando você quer estrutura, validação e segurança |


* chain of thought
* react
* tree of thought

## Agents Design
___

1. Defina os objetivos de forma clara; - `foque na tarefa e não no agente`
2. Defina como o agente irá coletar ou receber as informações;
3. Escolhe o agente a outros sistemas;
4. Integre o agente a outros sistemas;
5. Monitore e otimize;
6. Garanta a segurança e privacidade dos dados.

### Design de um agente - LANGCHAIN
___

* Modelo para tomada de decisão;
* Escolha as ferramentas;
* Escolha um sistema de memória embutido;
* Guardrails para limitar comportamento indesejado;
* Sistema de logging e observabilidade;

### Streaming in LangChain
___

In [97]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente especialista em IA."),
    ("human", "{pergunta}")
])

parser = StrOutputParser()

chain = prompt | llm | parser

chain.invoke({"pergunta": "Explique o que são redes neurais artificiais com um exemplo."})


'Redes neurais artificiais são modelos computacionais inspirados no funcionamento do cérebro humano, que são utilizados para realizar tarefas de aprendizado de máquina. Elas são compostas por neurônios artificiais interconectados em camadas, onde cada neurônio recebe entradas, realiza um cálculo e produz uma saída.\n\nUm exemplo de rede neural artificial é a rede neural convolucional (CNN), amplamente utilizada em tarefas de visão computacional, como reconhecimento de imagens. Neste tipo de rede, as camadas convolucionais são responsáveis por extrair características das imagens, enquanto as camadas de pooling reduzem a dimensionalidade dos dados. Por fim, as camadas totalmente conectadas realizam a classificação final da imagem.\n\nEm resumo, as redes neurais artificiais são poderosas ferramentas de aprendizado de máquina que podem ser aplicadas em uma variedade de problemas, desde reconhecimento de padrões até processamento de linguagem natural.'

In [98]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4.1", streaming=True)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente especialista em IA."),
    ("human", "{pergunta}")
])

parser = StrOutputParser()
chain = prompt | llm | parser

chunks =[]
for chunk in chain.stream({"pergunta": "Explique o que são redes neurais artificiais com um exemplo."}):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

print("\n\n")
print(chunks)
final_response = "".join(chunks)
print(final_response)

Redes neurais artificiais são sistemas computacionais inspirados no funcionamento do cérebro humano. Elas são formadas por unidades chamadas **neurônios artificiais** organizados em camadas. Esses neurônios recebem informações, processam e transmitem para outros neurônios, permitindo que a rede “aprenda” padrões em dados.

**Como funciona, de forma simples:**
- A camada de entrada recebe os dados (por exemplo, uma imagem).
- Os neurônios processam esses dados realizando operações matemáticas (como somas e ativações).
- A informação passa por camadas intermediárias (camadas ocultas), onde o conhecimento é refinado.
- Por fim, a camada de saída fornece o resultado (por exemplo, identificar se há um gato na imagem).

**Exemplo prático: Reconhecimento de dígitos escritos à mão**
Imagine que você quer que um computador reconheça números de 0 a 9 escritos à mão.

1. Você fornece imagens de dígitos para a rede neural.
2. Cada imagem é transformada em uma lista de números (os tons de cinza dos

Entender **quando usar streaming** é essencial para desenvolver aplicações mais eficientes, responsivas e amigáveis.

---

## 🧠 O que o streaming resolve?

### ✅ **Problema sem streaming:**

* Você envia o prompt para o modelo.
* O sistema **espera a resposta inteira** do LLM para depois exibir ou processar.
* Resultado: tempo de espera maior, experiência menos interativa.

### ✅ **Com streaming:**

* A resposta do modelo começa a ser **emitida token por token** (palavra por palavra).
* Você pode **mostrar, processar ou agir** conforme cada parte chega.
* Resultado: resposta **mais rápida e fluida**.

---

## 📦 Casos típicos onde **streaming é necessário (ou recomendado)**

### 1. **Chatbots e assistentes virtuais**

📲 **Exemplo**: Aplicativos como ChatGPT, interfaces com assistentes em tempo real.

* Exibir a resposta conforme o modelo "pensa".
* Torna a conversa mais **natural** e **responsiva**.

✅ Recomendado: melhora **UX** (experiência do usuário).

---

### 2. **Interfaces web ou mobile com atualização ao vivo**

🖥️ **Exemplo**: Interfaces React/Vue exibindo respostas em tempo real.

* Enquanto o modelo responde, o conteúdo já aparece no componente.
* Pode ser integrado com animações ou placeholders.

✅ Torna o app mais fluido, parece que o modelo está "digitando".

---

### 3. **Experiências de voz (text-to-speech)**

🎙️ **Exemplo**: um sistema que "fala" a resposta gerada.

* Assim que os primeiros tokens chegam, eles já podem ser passados para um sistema de voz.
* Isso reduz o tempo entre a pergunta e a resposta falada.

✅ Necessário para **naturalidade** em interações por voz.

---

### 4. **Carregamento progressivo em terminais e scripts CLI**

🧑‍💻 **Exemplo**: Chatbots de terminal, interfaces de linha de comando com LLM.

* Usuário vê o conteúdo aparecendo em tempo real (efeito "digitando...").
* Boa experiência em scripts interativos.

---

### 5. **Tarefas longas ou respostas extensas**

📚 **Exemplo**: Geração de textos longos, códigos, artigos, análises complexas.

* O modelo pode demorar vários segundos para finalizar.
* Com streaming, o usuário não fica esperando no escuro.

✅ Ideal para evitar **tempo de espera** percebido.

---

## ⚠️ Quando **não** é necessário usar streaming?

| Cenário                                                          | Use streaming?  |
| ---------------------------------------------------------------- | --------------- |
| Respostas muito curtas                                           | ❌ Desnecessário |
| Processamento em batch (várias chamadas de uma vez)              | ❌               |
| Uso em pipelines de backend com foco em velocidade total, não UX | ❌               |
| Quando vai salvar a resposta direto no banco ou arquivo          | ❌               |

---

## 🧪 Dica de ouro

Mesmo quando você **não precisa exibir streaming**, pode usá-lo para:

* Processar tokens à medida que chegam.
* Implementar validações ou cancelamentos precoces.
* Controlar o comportamento do modelo de forma mais granular.

---

In [99]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

completion = model.invoke("Hi there!")
# Hi!

completions = model.batch(["Hi there!", "Bye!"])
# ['Hi!', 'See you!']

for token in model.stream("Bye!"):
    print(token)
    # Good
    # bye
    # !

content='' additional_kwargs={} response_metadata={} id='run--8b916096-d536-4fd8-b8da-4f8621eea056'
content='Good' additional_kwargs={} response_metadata={} id='run--8b916096-d536-4fd8-b8da-4f8621eea056'
content='bye' additional_kwargs={} response_metadata={} id='run--8b916096-d536-4fd8-b8da-4f8621eea056'
content='!' additional_kwargs={} response_metadata={} id='run--8b916096-d536-4fd8-b8da-4f8621eea056'
content=' If' additional_kwargs={} response_metadata={} id='run--8b916096-d536-4fd8-b8da-4f8621eea056'
content=' you' additional_kwargs={} response_metadata={} id='run--8b916096-d536-4fd8-b8da-4f8621eea056'
content=' have' additional_kwargs={} response_metadata={} id='run--8b916096-d536-4fd8-b8da-4f8621eea056'
content=' any' additional_kwargs={} response_metadata={} id='run--8b916096-d536-4fd8-b8da-4f8621eea056'
content=' more' additional_kwargs={} response_metadata={} id='run--8b916096-d536-4fd8-b8da-4f8621eea056'
content=' questions' additional_kwargs={} response_metadata={} id='run-

In [100]:
from typing_extensions import Annotated, TypedDict
from pydantic import BaseModel, Field

class PydanticUserInfo(BaseModel):
    """User's info."""
    name: Annotated[str, Field(description="User's name. Defaults to ''", default=None)]
    country: Annotated[str, Field(description="Where the user lives. Defaults to ''", default=None, )]


In [101]:
llm_with_structure = llm.with_structured_output(PydanticUserInfo)

In [102]:
structured_output = llm_with_structure.invoke("The sky is blue")

In [103]:
structured_output

PydanticUserInfo(name='', country='')

In [104]:
print(structured_output.name)

In [105]:
from pydantic import BaseModel
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a estrutura esperada
class Pessoa(BaseModel):
    nome: str
    idade: int
    interesses: list[str]

# Cria o parser usando a classe Pessoa
parser = PydanticOutputParser(pydantic_object=Pessoa)

# Cria o template do prompt incluindo as instruções de formato
prompt = PromptTemplate(
    template="Extraia nome, idade e interesses do seguinte texto:\nTexto: {texto}\n{format_instructions}",
    input_variables=["texto"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Inicializa o modelo de linguagem
llm = ChatOpenAI(
    model="gpt-4o-mini",
)

# Cria a cadeia (chain) de processamento
chain = prompt | llm | parser

# Executa a extração
resposta = chain.invoke({"texto": "Joana tem 30 anos e gosta de leitura, yoga e viagens."})

print("Resultado:")
print(f"Nome: {resposta.nome}")
print(f"Idade: {resposta.idade}")
print(f"Interesses: {resposta.interesses}")
print(f"\nTipo do objeto: {type(resposta)}")
print(f"Objeto completo: {resposta}")


Resultado:
Nome: Joana
Idade: 30
Interesses: ['leitura', 'yoga', 'viagens']

Tipo do objeto: <class '__main__.Pessoa'>
Objeto completo: nome='Joana' idade=30 interesses=['leitura', 'yoga', 'viagens']


In [106]:
from pydantic import BaseModel, Field, validator
from typing import List
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a estrutura esperada com validações e descrições completas
class Pessoa(BaseModel):
    """Modelo para representar informações de uma pessoa extraídas de texto."""
    
    nome: str = Field(
        description="Nome completo da pessoa",
        min_length=2,
        max_length=100,
        example="Maria Silva"
    )
    
    idade: int = Field(
        description="Idade da pessoa em anos",
        ge=0,  # greater or equal (>= 0)
        le=150,  # less or equal (<= 150)
        example=30
    )
    
    interesses: List[str] = Field(
        description="Lista de hobbies, atividades ou interesses da pessoa",
        min_items=1,
        max_items=10,
        example=["leitura", "yoga", "viagens"]
    )
    
    # Validador customizado para normalizar o nome
    @validator('nome')
    def validar_nome(cls, v):
        if not v.strip():
            raise ValueError('Nome não pode estar vazio')
        # Capitaliza cada palavra do nome
        return ' '.join(word.capitalize() for word in v.strip().split())
    
    # Validador para normalizar interesses
    @validator('interesses')
    def validar_interesses(cls, v):
        if not v:
            raise ValueError('Deve haver pelo menos um interesse')
        # Remove duplicatas e normaliza para minúsculas
        interesses_limpos = []
        for interesse in v:
            interesse_limpo = interesse.strip().lower()
            if interesse_limpo and interesse_limpo not in interesses_limpos:
                interesses_limpos.append(interesse_limpo)
        return interesses_limpos
    
    class Config:
        # Configurações do modelo
        str_strip_whitespace = True  # Remove espaços em branco automaticamente
        validate_assignment = True   # Valida também durante atribuições
        schema_extra = {
            "example": {
                "nome": "Ana Costa",
                "idade": 28,
                "interesses": ["culinária", "fotografia", "natação"]
            }
        }

# Cria o parser usando a classe Pessoa
parser = PydanticOutputParser(pydantic_object=Pessoa)

# Template mais robusto
prompt = PromptTemplate(
    template="""
    Você é um especialista em extração de informações pessoais de textos.
    
    Extraia nome, idade e interesses do seguinte texto de forma precisa:
    
    TEXTO: {texto}
    
    INSTRUÇÕES IMPORTANTES:
    - Se a idade não estiver explícita, tente inferir com base no contexto
    - Para interesses, inclua hobbies, atividades, gostos pessoais mencionados
    - Seja preciso e não invente informações que não estão no texto
    
    {format_instructions}
    """,
    input_variables=["texto"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Inicializa o modelo de linguagem
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

# Cria a cadeia (chain) de processamento
chain = prompt | llm | parser

# Teste com diferentes exemplos
textos_teste = [
    "Joana tem 30 anos e gosta de leitura, yoga e viagens.",
    "Pedro é um jovem de 25 anos apaixonado por futebol, culinária e música eletrônica.",
    "Maria, 45 anos, adora jardinagem e pintura em aquarela."
]

print("=== RESULTADOS DA EXTRAÇÃO ===\n")

for i, texto in enumerate(textos_teste, 1):
    print(f"TESTE {i}:")
    print(f"Texto original: {texto}")
    
    try:
        resposta = chain.invoke({"texto": texto})
        print(f"✅ Extração bem-sucedida:")
        print(f"   Nome: {resposta.nome}")
        print(f"   Idade: {resposta.idade}")
        print(f"   Interesses: {resposta.interesses}")
        print(f"   Objeto validado: {resposta}")
        
    except Exception as e:
        print(f"❌ Erro na extração: {e}")
    
    print("-" * 50)

# Exemplo de como acessar o schema JSON gerado
print("\n=== SCHEMA JSON GERADO ===")
print(parser.get_format_instructions())

=== RESULTADOS DA EXTRAÇÃO ===

TESTE 1:
Texto original: Joana tem 30 anos e gosta de leitura, yoga e viagens.


/tmp/ipykernel_520984/179862137.py:33: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('nome')
/tmp/ipykernel_520984/179862137.py:41: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('interesses')
/home/fabiolima/Workdir/langchain/study_langchain/.venv/lib/python3.13/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_sche

✅ Extração bem-sucedida:
   Nome: Joana
   Idade: 30
   Interesses: ['leitura', 'yoga', 'viagens']
   Objeto validado: nome='Joana' idade=30 interesses=['leitura', 'yoga', 'viagens']
--------------------------------------------------
TESTE 2:
Texto original: Pedro é um jovem de 25 anos apaixonado por futebol, culinária e música eletrônica.
✅ Extração bem-sucedida:
   Nome: Pedro
   Idade: 25
   Interesses: ['futebol', 'culinária', 'música eletrônica']
   Objeto validado: nome='Pedro' idade=25 interesses=['futebol', 'culinária', 'música eletrônica']
--------------------------------------------------
TESTE 3:
Texto original: Maria, 45 anos, adora jardinagem e pintura em aquarela.
✅ Extração bem-sucedida:
   Nome: Maria
   Idade: 45
   Interesses: ['jardinagem', 'pintura em aquarela']
   Objeto validado: nome='Maria' idade=45 interesses=['jardinagem', 'pintura em aquarela']
--------------------------------------------------

=== SCHEMA JSON GERADO ===
The output should be formatted as a J

## LCEL - LangChain Expression Language 
___

`A declarative way to compose AI workflows`

## LangGraph

A framework for `agentic workgflows` with complex state management.

Vamos explorar a ideia da **interface `Runnable` no LangChain**, que é uma das peças centrais da **LangChain Expression Language (LCEL)**.

---

## 🧠 O que é `Runnable`?

Em termos simples:

> **`Runnable` é uma interface padrão que define como um componente da LangChain pode ser executado.**

Tudo o que pode ser **invocado, encadeado ou executado** — como um modelo, um prompt, um parser — implementa a interface `Runnable`.

---

## 📦 Por que isso é importante?

A interface `Runnable` é o que permite você fazer coisas como:

```python
prompt | llm | parser
```

Cada componente aqui (`prompt`, `llm`, `parser`) é um **Runnable** — eles sabem **como receber entrada e produzir saída** de forma padronizada.

---

## 🧩 Métodos principais de `Runnable`

| Método           | O que faz                                                   |
| ---------------- | ----------------------------------------------------------- |
| `invoke(input)`  | Executa uma única chamada sincrônica                        |
| `stream(input)`  | Executa com **streaming**, útil para chat/respostas ao vivo |
| `batch(inputs)`  | Executa várias entradas de uma vez                          |
| `ainvoke(input)` | Versão assíncrona de `invoke()`                             |
| `astream(input)` | Versão assíncrona de `stream()`                             |
| `abatch(inputs)` | Versão assíncrona de `batch()`                              |

---

## 🔧 Exemplo didático

```python
from langchain_core.runnables import RunnableLambda

# Cria um componente que dobra o valor de entrada
dobro = RunnableLambda(lambda x: x * 2)

print(dobro.invoke(10))  # → 20
```

Esse `RunnableLambda` é uma função simples transformada num `Runnable`.

---

## 📐 Por que isso é poderoso?

Porque você pode **compor componentes diferentes** como se fossem "filtros de dados" — tipo Unix Pipes (`|`).

Por exemplo:

```python
chain = prompt | llm | parser
```

Internamente, isso é equivalente a:

```python
output_prompt = prompt.invoke(inputs)
output_llm = llm.invoke(output_prompt)
output_final = parser.invoke(output_llm)
```

---

## 🤖 Todo LLM, prompt, parser ou chain em LCEL é um `Runnable`

### Exemplo:

* `ChatPromptTemplate` → `Runnable`: recebe `dict` → gera prompt formatado.
* `ChatOpenAI` → `Runnable`: recebe prompt → gera resposta.
* `StrOutputParser` → `Runnable`: recebe resposta → extrai string.

---

## 💡 Curiosidade: você pode criar suas próprias chains

```python
from langchain_core.runnables import RunnableLambda

formatador = RunnableLambda(lambda x: f"Olá, {x}!")
maiusculo = RunnableLambda(lambda x: x.upper())

saudacao_chain = formatador | maiusculo

print(saudacao_chain.invoke("fabio"))  # → OLÁ, FABIO!
```

Simples, poderoso e reutilizável.

---

## 🧠 Resumo final

A interface `Runnable` é:

✅ O **coração do encadeamento** em LangChain
✅ Uma maneira de garantir que **todos os componentes se comportem de forma previsível**
✅ Uma forma de criar **pipelines reutilizáveis e composáveis**
✅ O que permite usar métodos como `.invoke()`, `.stream()`, `.batch()` em qualquer etapa da sua chain

---

Se quiser, posso montar um gráfico tipo "pipeline visual" explicando como os Runnables se conectam.

Ou gerar um notebook com exemplos didáticos de `RunnableLambda`, `RunnableMap`, `Prompt | LLM | Parser`.


In [109]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Componentes base
prompt = ChatPromptTemplate.from_template("Me dê uma explicação simples sobre {topic}")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # Use o modelo mais novo disponível
parser = StrOutputParser()

# Encadeamento moderno com LCEL (LangChain Expression Language)
chain = prompt | llm | parser

# Executa batch de múltiplas entradas
result = chain.batch([
    {'topic': 'python'},
    {'topic': 'machine learning'}
])

for item in result:
    print(item)


Python é uma linguagem de programação de alto nível, muito popular por sua simplicidade e legibilidade. Ela foi criada por Guido van Rossum e lançada pela primeira vez em 1991. Aqui estão alguns pontos principais sobre Python:

1. **Fácil de Aprender**: A sintaxe do Python é clara e intuitiva, o que a torna uma ótima escolha para iniciantes.

2. **Versátil**: Python pode ser usado para uma ampla variedade de aplicações, incluindo desenvolvimento web, análise de dados, inteligência artificial, automação de tarefas, e muito mais.

3. **Bibliotecas e Frameworks**: Python possui uma vasta coleção de bibliotecas e frameworks que facilitam o desenvolvimento. Por exemplo, o Django é usado para desenvolvimento web, enquanto o Pandas é popular para análise de dados.

4. **Comunidade Ativa**: Python tem uma grande comunidade de desenvolvedores, o que significa que há muitos recursos, tutoriais e suporte disponíveis.

5. **Multiplataforma**: Python pode ser executado em diferentes sistemas operac

In [ ]:
# Exemplos Práticos: Imperative Composition vs LCEL
# Certifique-se de ter instalado: pip install langchain langchain-openai

import os
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_core.output_parsers import StrOutputParser

# Configure sua API key da OpenAI
# os.environ["OPENAI_API_KEY"] = "sua-api-key-aqui"

# ============================================================================
# EXEMPLO 1: TRADUTOR SIMPLES
# ============================================================================

print("=== EXEMPLO 1: TRADUTOR SIMPLES ===\n")

# Componentes base
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
translator_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um tradutor especializado. Traduza o texto para {idioma}."),
    ("human", "{texto}")
])

# VERSÃO IMPERATIVE COM @chain
@chain
def tradutor_imperativo(values):
    """Tradutor usando composição imperativa"""
    prompt = translator_template.invoke(values)
    response = model.invoke(prompt)
    return response.content

# VERSÃO LCEL
tradutor_lcel = translator_template | model | StrOutputParser()

# TESTANDO AMBOS
texto_teste = "Hello, how are you today?"
idioma_teste = "português"

print("🔧 IMPERATIVO:")
resultado_imp = tradutor_imperativo.invoke({
    "texto": texto_teste, 
    "idioma": idioma_teste
})
print(f"Resultado: {resultado_imp}\n")

print("⚡ LCEL:")
resultado_lcel = tradutor_lcel.invoke({
    "texto": texto_teste, 
    "idioma": idioma_teste
})
print(f"Resultado: {resultado_lcel}\n")

# ============================================================================
# EXEMPLO 2: ANALISADOR DE SENTIMENTOS COM LÓGICA CONDICIONAL
# ============================================================================

print("=== EXEMPLO 2: ANALISADOR DE SENTIMENTOS ===\n")

sentiment_template = ChatPromptTemplate.from_messages([
    ("system", "Analise o sentimento do texto: POSITIVO, NEGATIVO ou NEUTRO. Responda apenas com uma palavra."),
    ("human", "{texto}")
])

response_template = ChatPromptTemplate.from_messages([
    ("system", "Com base no sentimento {sentimento}, gere uma resposta {tipo_resposta} ao texto original."),
    ("human", "Texto original: {texto}")
])

# VERSÃO IMPERATIVE COM LÓGICA CONDICIONAL
@chain
def analisador_imperativo(values):
    """Analisador com lógica condicional complexa"""
    texto = values["texto"]
    
    # Passo 1: Analisar sentimento
    sentiment_prompt = sentiment_template.invoke({"texto": texto})
    sentimento = model.invoke(sentiment_prompt).content.strip().upper()
    
    # Passo 2: Lógica condicional baseada no sentimento
    if sentimento == "POSITIVO":
        tipo_resposta = "encorajadora e positiva"
    elif sentimento == "NEGATIVO":
        tipo_resposta = "empática e de apoio"
    else:
        tipo_resposta = "equilibrada e informativa"
    
    # Passo 3: Gerar resposta personalizada
    response_prompt = response_template.invoke({
        "sentimento": sentimento,
        "tipo_resposta": tipo_resposta,
        "texto": texto
    })
    
    resposta_final = model.invoke(response_prompt).content
    
    # Retorna resultado estruturado
    return {
        "texto_original": texto,
        "sentimento_detectado": sentimento,
        "tipo_resposta": tipo_resposta,
        "resposta_gerada": resposta_final
    }

# VERSÃO LCEL (mais limitada para lógica condicional)
analisador_lcel_simples = sentiment_template | model | StrOutputParser()

# TESTANDO O ANALISADOR COMPLEXO
textos_teste = [
    "Estou muito feliz com meu novo emprego!",
    "Estou muito triste e desanimado hoje...",
    "O clima está nublado hoje."
]

print("🔧 IMPERATIVO (com lógica condicional):")
for texto in textos_teste:
    resultado = analisador_imperativo.invoke({"texto": texto})
    print(f"📝 Texto: {resultado['texto_original']}")
    print(f"😊 Sentimento: {resultado['sentimento_detectado']}")
    print(f"🎯 Tipo de resposta: {resultado['tipo_resposta']}")
    print(f"💬 Resposta: {resultado['resposta_gerada']}")
    print("-" * 50)

print("\n⚡ LCEL (apenas análise de sentimento):")
for texto in textos_teste:
    sentimento = analisador_lcel_simples.invoke({"texto": texto})
    print(f"📝 '{texto}' → 😊 {sentimento}")

# ============================================================================
# EXEMPLO 3: PIPELINE DE PROCESSAMENTO DE TEXTO
# ============================================================================

print("\n=== EXEMPLO 3: PIPELINE DE PROCESSAMENTO ===\n")

# Templates para o pipeline
extractor_template = ChatPromptTemplate.from_messages([
    ("system", "Extraia as palavras-chave principais do texto. Liste apenas as palavras separadas por vírgula."),
    ("human", "{texto}")
])

summarizer_template = ChatPromptTemplate.from_messages([
    ("system", "Resuma o texto em uma frase focando nas palavras-chave: {keywords}"),
    ("human", "{texto}")
])

# VERSÃO IMPERATIVE
@chain
def pipeline_imperativo(values):
    """Pipeline completo de processamento"""
    texto = values["texto"]
    
    # Etapa 1: Extrair palavras-chave
    keywords_prompt = extractor_template.invoke({"texto": texto})
    keywords = model.invoke(keywords_prompt).content.strip()
    
    # Etapa 2: Resumir com base nas keywords
    summary_prompt = summarizer_template.invoke({
        "texto": texto,
        "keywords": keywords
    })
    summary = model.invoke(summary_prompt).content
    
    return {
        "texto_original": texto,
        "palavras_chave": keywords,
        "resumo": summary
    }

# VERSÃO LCEL com RunnablePassthrough para dados complexos
from langchain_core.runnables import RunnablePassthrough

def extrair_keywords(values):
    """Função auxiliar para extrair keywords"""
    prompt = extractor_template.invoke(values)
    keywords = model.invoke(prompt).content.strip()
    return {**values, "keywords": keywords}

pipeline_lcel = (
    RunnablePassthrough() 
    | extrair_keywords 
    | summarizer_template 
    | model 
    | StrOutputParser()
)

# TESTE DO PIPELINE
texto_pipeline = """
A inteligência artificial está transformando rapidamente diversos setores da economia. 
Empresas estão investindo pesadamente em machine learning e automação para melhorar 
a eficiência operacional. No entanto, surgem preocupações sobre o impacto no emprego 
e a necessidade de regulamentação adequada para garantir o uso ético da tecnologia.
"""

print("🔧 PIPELINE IMPERATIVO:")
resultado_pipeline = pipeline_imperativo.invoke({"texto": texto_pipeline})
print(f"📝 Original: {resultado_pipeline['texto_original'][:100]}...")
print(f"🔑 Keywords: {resultado_pipeline['palavras_chave']}")
print(f"📄 Resumo: {resultado_pipeline['resumo']}\n")

print("⚡ PIPELINE LCEL:")
resumo_lcel = pipeline_lcel.invoke({"texto": texto_pipeline})
print(f"📄 Resumo: {resumo_lcel}")

# ============================================================================
# EXEMPLO 4: CHATBOT COM MEMÓRIA (IMPERATIVO)
# ============================================================================

print("\n=== EXEMPLO 4: CHATBOT COM HISTÓRICO ===\n")

# Este exemplo mostra como o imperativo é melhor para estados complexos
@chain
def chatbot_com_memoria(values):
    """Chatbot que mantém contexto da conversa"""
    pergunta_atual = values["pergunta"]
    historico = values.get("historico", [])
    
    # Constrói o contexto com histórico
    messages = [("system", "Você é um assistente útil que lembra da conversa anterior.")]
    
    # Adiciona histórico
    for h in historico:
        messages.append(("human", h["pergunta"]))
        messages.append(("assistant", h["resposta"]))
    
    # Adiciona pergunta atual
    messages.append(("human", pergunta_atual))
    
    # Cria template dinâmico
    template_dinamico = ChatPromptTemplate.from_messages(messages)
    
    # Gera resposta
    prompt = template_dinamico.invoke({})
    resposta = model.invoke(prompt).content
    
    # Atualiza histórico
    novo_historico = historico + [{
        "pergunta": pergunta_atual,
        "resposta": resposta
    }]
    
    return {
        "resposta": resposta,
        "historico_atualizado": novo_historico
    }

# SIMULAÇÃO DE CONVERSA
print("🤖 CHATBOT COM MEMÓRIA:")
historico_conversa = []

perguntas = [
    "Qual é a capital do Brasil?",
    "Quantos habitantes ela tem aproximadamente?",
    "Qual é o principal ponto turístico dessa cidade?"
]

for pergunta in perguntas:
    resultado = chatbot_com_memoria.invoke({
        "pergunta": pergunta,
        "historico": historico_conversa
    })
    
    print(f"👤 Você: {pergunta}")
    print(f"🤖 Bot: {resultado['resposta']}")
    print()
    
    historico_conversa = resultado["historico_atualizado"]

print("=" * 60)
print("🎯 RESUMO DAS DIFERENÇAS:")
print("• IMPERATIVO: Melhor para lógica complexa, estados, condicionais")
print("• LCEL: Melhor para pipelines lineares, performance, simplicidade")
print("• Use @chain para combinar o melhor dos dois mundos!")
print("=" * 60)

### 1. Conceito Fundamental: Runnable


O Runnable é a interface base do LangChain que padroniza como diferentes componentes podem ser executados e combinados. Qualquer objeto que implementa Runnable possui métodos como .invoke(), .stream(), .batch(), etc.

In [ ]:
# exemplo
# Todos estes são objetos Runnable:
translator_template = ChatPromptTemplate.from_messages([...])  # Runnable
model = ChatOpenAI(model="gpt-4.1", temperature=0.0)          # Runnable  
parser_output = StrOutputParser()                             # Runnable

### 2. RunnableSequence(operador pipe | )

Quando você usa o `(operador |)` , está criando uma RunnableSequence - uma cadeia onde a saída de um componente vira entrada do próximo.

```python
translator_chain = translator_template | model | parser_output
# Isso é equivalente a:
# RunnableSequence(first=translator_template, middle=[model], last=parser_output)
```

Input: {"texto": "Hello...", "idioma": "português"}
  ↓
translator_template → Gera prompt formatado
  ↓
model → Processa prompt e gera resposta
  ↓
parser_output → Extrai string da resposta
  ↓
Output: "Olá, como você está hoje?"